In [1]:

from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from transformers import BertTokenizerFast, BertForSequenceClassification, AdamW, BertModel

import logging
import os
import re
import sys

import pandas as pd
import torch


embed_size = 300
max_len = 628



train = pd.read_csv(r"F:\CodeFiled\NLP\Kaggle1_but_glove\corpus\imdb\labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
test = pd.read_csv(r"F:\CodeFiled\NLP\Kaggle1_but_glove\corpus\imdb\testData.tsv", header=0, delimiter="\t", quoting=3)


class TrainDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


class TestDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, num_samples=0):
        self.encodings = encodings
        self.num_samples = num_samples

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return self.num_samples


def get_chunk_article(directory):
    df = pd.DataFrame()

    for filename in os.listdir(directory):

        if filename.endswith('.txt'):
            with open(os.path.join(directory, filename), encoding='utf-8') as file:
                content = file.read()
            df[filename] = [content]

    df = df.T
    new_column_names = {
        0: 'article',
    }

    df.rename(columns=new_column_names, inplace=True)
    return df


def read_chunk_files(directory, label ):
    dataframes = []
    for i,filename in enumerate(os.listdir(directory)):
        filepath = os.path.join(directory, filename)
        dataframes = get_chunk_article(filepath)
        if label != 2:
            dataframes['labels'] = label
        break
    for i,filename in enumerate(os.listdir(directory)):
        if i == 0:
            continue
        if filename.startswith("chunk_"):
            filepath = os.path.join(directory, filename)
            df = get_chunk_article(filepath)
            if label != 2:
                df['labels'] = label
            dataframes = pd.concat([dataframes,df])


    return dataframes

# 定义文件夹路径
dir_train_po = '2017depression/cleaned-training/cleaned_positive'
dir_train_ne = '2017depression/cleaned-training/cleaned_negative'
dir_test = '2017depression/cleaned-testing'

# 读取正例和负例的chunk文件并添加标签
train_positive = read_chunk_files(dir_train_po, 1)
train_negative = read_chunk_files(dir_train_ne, 0)
test = read_chunk_files(dir_test,2)

# 合并为train文件
train = pd.concat([train_positive, train_negative])

# 输出合并后的train和test文件
print("Train Dataset:")
print(train.head())
print("\nTest Dataset:")
print(test.head())




C:\Users\FlashBlack7\AppData\Local\Temp\ipykernel_16100\1497496319.py:58: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[filename] = [content]
C:\Users\FlashBlack7\AppData\Local\Temp\ipykernel_16100\1497496319.py:58: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[filename] = [content]
C:\Users\FlashBlack7\AppData\Local\Temp\ipykernel_16100\1497496319.py:58: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all co

Train Dataset:
                                                                   article  \
train_subject1095_1.txt   The last line is pretty much the same as the ...   
train_subject1190_1.txt   I'm sure I'll get flamed for this, but true f...   
train_subject1191_1.txt               Hi Ruben! Happy Birthday! \n\n\n\n\n   
train_subject1199_1.txt   In the US, the Supreme Court has ruled that i...   
train_subject1201_1.txt   lol I do that on a regular basis. ;) \n\n\n\n...   

                         labels  
train_subject1095_1.txt       1  
train_subject1190_1.txt       1  
train_subject1191_1.txt       1  
train_subject1199_1.txt       1  
train_subject1201_1.txt       1  

Test Dataset:
                                                           article
subject1005.txt   "Kill Bill Siren" was awesome. These seriousl...
subject1062.txt   I know that wind chill is used when it is col...
subject1085.txt   Exactly! I don't know how you can get a job a...
subject1103.txt   Don't tell 

In [2]:

program = os.path.basename(sys.argv[0])
logger = logging.getLogger(program)

logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
logging.root.setLevel(level=logging.INFO)
logger.info(r"running %s" % ''.join(sys.argv))

# train_texts, train_labels, test_texts = [], [], []
# for i, review in enumerate(train["review"]):
#     train_texts.append(review)
#     train_labels.append(train['sentiment'][i])
#
# for review in test['review']:
#     test_texts.append(review)
#
# train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)
BERT_PATH = r'F:\CodeFiled\NLP\pretrainedModels\uncased_L-12_H-768_A-12'

tokenizer = BertTokenizerFast.from_pretrained(BERT_PATH)



bert = BertModel.from_pretrained(BERT_PATH)

print('load bert model over')
# train_encodings = tokenizer(train_texts, truncation=True, padding=True)
# val_encodings = tokenizer(val_texts, truncation=True, padding=True)
# test_encodings = tokenizer(test_texts, truncation=True, padding=True)
#
# train_dataset = TrainDataset(train_encodings, train_labels)
# val_dataset = TrainDataset(val_encodings, val_labels)
# test_dataset = TestDataset(test_encodings, num_samples=len(test_texts))

# temp = tokenizer.tokenize(train['article'][0])

temp = train['article'][0]


2024-04-18 12:14:47,361: INFO: running D:\SoftWare\Anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py-fC:\Users\FlashBlack7\AppData\Roaming\jupyter\runtime\kernel-364e6772-e04d-4758-a93c-7cdc6fc8b6c1.json


load bert model over


In [3]:
# encoding = tokenizer.encode_plus(
#     train['article'][5],
#     max_length=790,
#     padding='max_length',
#     truncation=True,
#     return_tensors='pt'
# )['input_ids']
# type(tokenizer.encode(train['article'][5], add_special_tokens=True))

In [4]:
len_cut =790  # token平均长度中位数
encoded_train_datas = list()
for i in range(train.shape[0]):

   encoded_train_datas.append(tokenizer.encode_plus(
        train['article'][i],
        max_length=len_cut,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )['input_ids'].squeeze().tolist())


train_bert = pd.DataFrame({'feature':encoded_train_datas,'labels':train['labels']})

In [5]:
#coding=utf-8

import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd



home_path='bert_trad_train.csv'

train_bert.to_csv(home_path,index=False,sep=',')  #windows下的路径



In [12]:
print(type(train_bert['feature'][0]))


<class 'list'>


In [7]:
# tokenizer.encode(train['article'][5], add_special_tokens=True)

In [8]:
len_cut =790  # token平均长度中位数
encoded_train_datas = list()
for i in range(train.shape[0]):
   encoded_train_datas.append(tokenizer.encode(train['article'][i], add_special_tokens=True)[:790])

train_bert = pd.DataFrame({'feature':encoded_train_datas,'labels':train['labels']})

In [9]:
#coding=utf-8

import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd



home_path='bert_xgboost_train.csv'

train_bert.to_csv(home_path,index=False,sep=',')  #windows下的路径

